In [127]:
import sys
import nltk
import sklearn
import pandas
import numpy

In [ ]:
print("Python: {}".format(sys.version))
print("NLTK: {}".format(nltk.__version__))
print("Sklearn: {}".format(sklearn.__version__))
print("pandas: {}".format(pandas.__version__))
print("numpy: {}".format(numpy.__version__))

## 1. Load the Dataset

In [ ]:
import pandas as pd
import numpy as np

# load the sms
df = pd.read_table('SMSSpamCollection', header=None, encoding='utf-8')

In [ ]:
# print useful data
print(df.info())
print(df.head())

In [ ]:
# check the class distribution
classes = df[0]
print(classes.value_counts())

## 2. Preprocess the data

In [ ]:
# convert class labels to binary values 0=ham, 1=spam
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

print(classes[:10])
print(Y[:10])

In [ ]:
#store the sms message data
text_messages = df[1]
print(text_messages[:10])

In [ ]:
# use regular expression to replace email, urls,

# replace email address with 'emailaddr'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddr')

# replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}{/\S*}?$', 'webaddress')

# replace money symbols with 'moneysymb'
processed = processed.str.replace(r'�|\$','moneysymb')

# replcae 10 digit phone numbers with 'phone number'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')

# replace normal numbers with numbr
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

In [ ]:
# replace punc
processed = processed.str.replace(r'[\n\d\s]', ' ')

#replace whitespace
processed = processed.str.replace(r'\s+', ' ')

# remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [ ]:
# changing the words to lowercase
processed = processed.str.lower()
print(processed[:10])

In [ ]:
# remove stop words
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [ ]:
# remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [ ]:
from nltk.tokenize import word_tokenize

# creating a bag-of-words
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [ ]:
# print total no of words and print most common
print("Number of words:: {}".format(len(all_words)))
print("Most common words:: {}".format(all_words.most_common(15)))

In [ ]:
# use the first 1500 most common words as features
word_features = list(all_words.keys())[:1500]

In [ ]:
# define find_features function
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features

features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print(key)

In [ ]:
# find features for all messages
messages = zip(processed, Y)

# define a seed for reproducability
seed = 1
np.random.seed = seed
#np.random.shuffle(messages)

# call find_features for each sms messages
featuresets = [(find_features(text), label) for (text, label) in messages]

In [ ]:
# split training and testing data sets using sklearn
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresets, test_size=0.25, random_state= seed)


In [ ]:
print('Training {}'.format(len(training)))
print('Testing {}'.format(len(testing)))

## 3. Scikit-Learn Classifiers with NLTK

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
# define the model to train
names = ['K Nearest Neighbors', 'Decision Tree',
        'Random Forest', 'Logistic Regression',
        'SGD Classifier', 'Naive Bayes', 
        'SVN Linear']

classifier = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel='linear')
]
#models = zip(names, classifier)

In [ ]:
list(models)

In [126]:
# wrap models in NLTK
from nltk.classify.scikitlearn import SklearnClassifier

for (name, model) in zip(names, classifier):
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing) * 100
    print('{}: Accuracy: {}'.format(name, accuracy))


K Nearest Neighbors: Accuracy: 93.3237616654702
Decision Tree: Accuracy: 97.20028715003589
Random Forest: Accuracy: 98.06173725771716


C:\Users\Kapil Pawar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression: Accuracy: 98.7078248384781


C:\Users\Kapil Pawar\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGD Classifier: Accuracy: 97.98994974874373
Naive Bayes: Accuracy: 98.34888729361091
SVN Linear: Accuracy: 98.56424982053123


In [ ]:
# ensamble method - Voting Classifier
from sklearn.ensemble import VotingClassifier

models = zip(names, classifier)

In [125]:
nltk_ensamble = SklearnClassifier(VotingClassifier(estimators=list(zip(names, classifier)), voting='hard', n_jobs=-1))
nltk_ensamble.train(training)
accuracy = nltk.classify.accuracy(nltk_ensamble, testing) * 100
print("Ensamble Method Accuracy: {}".format(accuracy))

Ensamble Method Accuracy: 98.63603732950466
